In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# so that we can import `tools.py`
# Navigate two directories up from the current working directory
module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
# Add this directory to sys.path if it's not already included
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pickle
from tqdm import tqdm

In [3]:
%%time
# loading a sample from the dataset specified in config, in this case 'HuggingFaceFW/fineweb'
from tools import get_data_loaders
train_data_loader, val_data_loader = get_data_loaders(
    'HuggingFaceFW/fineweb', 
    batch_size=100, 
    streaming=True,
    subset_name = "CC-MAIN-2024-10" # using a different subset of fineweb from what we'll be training the model on
)

Resolving data files:   0%|          | 0/23781 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/250 [00:00<?, ?it/s]

CPU times: user 7.33 s, sys: 2.4 s, total: 9.73 s
Wall time: 16.2 s


In [4]:
%%time
batch = next(train_data_loader)
print(len(batch), batch[0])

100 That is to say, it’s a major departure from the authoritative air and hard-edged beauty of Singapore – in the best possible way.
The most eventful thing we did in Manila was buy our suits for Trevor’s wedding. I’ll get to that in a follow-up post. But there were some remarkable charms to this place, despite its rough exterior.
For instance, we stayed at a great little hotel/hostel, about a quarter mile from the ocean, called the Buoy. The owner was a domineering asshole, but fortunately we only met him once. His staff, on the other hand, were a delight. The all-female staff of the attached restaurant/bar were especially sweet and great to be around, and this led us to favor the hotel for our near-constant drinking during the few days we were there. We’d sit and drink buckets of beer, and work on this stupid little website we have (maybe you’ve heard of it), and play pool, and hang out with the Buoy girls. We spent a lot of time that way. (Hi girls!)
I’m pleased to report that we al

In [5]:
# turn it into one string instead of a list of strings
combined_string = '\n\n'.join(batch)

# Convert the string to bytes
combined_bytes = combined_string.encode('utf-8')
print(len(combined_bytes))

print(combined_string[:100])
print([b for b in combined_bytes[:100]])
print(combined_bytes.decode('utf-8')[:100])

310776
That is to say, it’s a major departure from the authoritative air and hard-edged beauty of Singapore
[84, 104, 97, 116, 32, 105, 115, 32, 116, 111, 32, 115, 97, 121, 44, 32, 105, 116, 226, 128, 153, 115, 32, 97, 32, 109, 97, 106, 111, 114, 32, 100, 101, 112, 97, 114, 116, 117, 114, 101, 32, 102, 114, 111, 109, 32, 116, 104, 101, 32, 97, 117, 116, 104, 111, 114, 105, 116, 97, 116, 105, 118, 101, 32, 97, 105, 114, 32, 97, 110, 100, 32, 104, 97, 114, 100, 45, 101, 100, 103, 101, 100, 32, 98, 101, 97, 117, 116, 121, 32, 111, 102, 32, 83, 105, 110, 103, 97, 112, 111]
That is to say, it’s a major departure from the authoritative air and hard-edged beauty of Singapore


# Regex
this is a pre-processing stage where we set the rules for what types of characters are allowed to be merged together

In [6]:
import regex as re

In [7]:
# don't ask me the specifics of how this plays out, i just know it's what they used for GPT4
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
# if you want to mess around with building your own tokenizer, then ^this string is one of the things to mess around with

In [8]:
compiled_pattern = re.compile(GPT4_SPLIT_PATTERN, re.UNICODE)
print(compiled_pattern)

regex.Regex("'(?i:[sdmt]|ll|ve|re)|[^\\r\\n\\p{L}\\p{N}]?+\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]++[\\r\\n]*|\\s*[\\r\\n]|\\s+(?!\\S)|\\s+", flags=regex.V0)


In [9]:
# split the text up into text chunks
text_chunks = re.findall(compiled_pattern, combined_string)
print(len(combined_string), len(text_chunks))

308521 59869


In [9]:
print(combined_string[:100])
print(text_chunks[:100])

|Viewing Single Post From: Spoilers for the Week of February 11th|
|Lil||Feb 1 2013, 09:58 AM|
Don't
['|Viewing', ' Single', ' Post', ' From', ':', ' Spoilers', ' for', ' the', ' Week', ' of', ' February', ' ', '11', 'th', '|\n', '|Lil', '||', 'Feb', ' ', '1', ' ', '201', '3', ',', ' ', '09', ':', '58', ' AM', '|\n', 'Don', "'t", ' care', ' about', ' Chloe', '/Taniel', '/Jen', '-Jen', '.', ' Don', "'t", ' care', ' about', ' Sami', ',', ' really', ',', ' but', ' hoping', ' that', ' we', ' get', ' some', ' good', ' "', 'SAMANTHA', ' GENE', '!!"', ' Marlena', ' Death', '-Stares', ' out', ' of', ' it', '.', ' And', ' "', 'newfound', '"', ' feelings', '.', ' Please', '.', ' If', ' only', '.\n', 'STEFANO', '!!', ' STEFANO', ',', ' STEFANO', ',', ' STEFANO', '!!!!', ' :', 'cheer', ':\n', '|Spoilers', ' for', ' the', ' Week', ' of', ' February', ' ', '11', 'th', ' ·', ' DAYS', ':', ' News']


In [10]:
# input text preprocessing
ids = [list(ch.encode("utf-8")) for ch in text_chunks] 
ids_backup = ids # saving this for later just to see how much compression we get
print(len(ids), ids[:100])

150958 [[124, 86, 105, 101, 119, 105, 110, 103], [32, 83, 105, 110, 103, 108, 101], [32, 80, 111, 115, 116], [32, 70, 114, 111, 109], [58], [32, 83, 112, 111, 105, 108, 101, 114, 115], [32, 102, 111, 114], [32, 116, 104, 101], [32, 87, 101, 101, 107], [32, 111, 102], [32, 70, 101, 98, 114, 117, 97, 114, 121], [32], [49, 49], [116, 104], [124, 10], [124, 76, 105, 108], [124, 124], [70, 101, 98], [32], [49], [32], [50, 48, 49], [51], [44], [32], [48, 57], [58], [53, 56], [32, 65, 77], [124, 10], [68, 111, 110], [39, 116], [32, 99, 97, 114, 101], [32, 97, 98, 111, 117, 116], [32, 67, 104, 108, 111, 101], [47, 84, 97, 110, 105, 101, 108], [47, 74, 101, 110], [45, 74, 101, 110], [46], [32, 68, 111, 110], [39, 116], [32, 99, 97, 114, 101], [32, 97, 98, 111, 117, 116], [32, 83, 97, 109, 105], [44], [32, 114, 101, 97, 108, 108, 121], [44], [32, 98, 117, 116], [32, 104, 111, 112, 105, 110, 103], [32, 116, 104, 97, 116], [32, 119, 101], [32, 103, 101, 116], [32, 115, 111, 109, 101], [32, 103, 11

so this regex just splits the text up into all the token ids that are allowed to be merged, meaning that the regex output we saw above is an upper limit on the tokens that we could end up with if we get a large enough vocabulary, rather than a starting point. 

# BPE tokenization

In [11]:
vocab_size = 1024 - 3 # the desired final vocabulary size. -3 for the three special tokens bos, eos, & pad
num_merges = vocab_size - 256

In [12]:
def get_stats(ids, counts=None):
    """
    Given a list of integers, return a dictionary of counts of consecutive pairs
    Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
    Optionally allows to update an existing dictionary of counts
    """
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]): # iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    """
    In the list of integers (ids), replace all consecutive occurrences
    of pair with the new integer token idx
    Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
    """
    newids = []
    i = 0
    while i < len(ids):
        # if not at the very last position AND the pair matches, replace it
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [13]:
# now let's actually do it
merges = {} # (int, int) -> int
for i in tqdm(range(num_merges)):
    # count the number of times every consecutive pair appears
    stats = {}
    for chunk_ids in ids:
        # passing in stats will update it in place, adding up counts
        get_stats(chunk_ids, stats)
    # find the pair with the highest count
    pair = max(stats, key=stats.get)
    # mint a new token: assign it the next available id
    idx = 256 + i
    # replace all occurrences of pair in ids with idx
    ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
    # save the merge
    merges[pair] = idx
    #print(f"merge {i+1}/{num_merges}: {pair} -> {idx} had {stats[pair]} occurrences")

100%|█████████████████████████████████| 765/765 [02:20<00:00,  5.44it/s]


In [14]:
og = sum([len(t) for t in ids_backup])
new = sum([len(t) for t in (ids)])
print("original length:", og)
print("ids length:", new)
print(f"compression ratio: {og / new:.2f}X")

original length: 735934
ids length: 301100
compression ratio: 2.44X


In [15]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    # given ids (list of integers), return Python string
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

print(ids[0], decode(ids[0]))

[124, 86, 813, 276] |Viewing


In [16]:
def encode(text):
    # given a string, return list of integers (the tokens)
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break # nothing else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(encode("Once upon a time,"))

[79, 969, 498, 261, 257, 602, 44]


In [19]:
# Ensure the tokenizers directory exists
if not os.path.exists('./models'):
    os.makedirs('./models')

# Prepare the tokenizer data to be saved
tokenizer_data = {'merges': merges}

# Save the tokenizer data using pickle
with open(f'./models/{vocab_size}.model', 'wb') as f:
    pickle.dump(tokenizer_data, f)

In [20]:
# taking a pre-existing tokenizer and trimming it down to a smaller size
# i basically ran this cell and then one above it multiple times until i got to the smallest possible size (512 - 3 = 509)
vocab_size = ((vocab_size + 3) // 2) - 3 # the -3's account for our special tokens
merges = {k: v for k, v in merges.items() if v < vocab_size}

In [21]:
from tokenizer import load_tokenizer_data, BPE_Tokenizer
vocab_size = 1024
tokenizer_data = load_tokenizer_data(f'models/{vocab_size-3}.model')
tokenizer = BPE_Tokenizer(tokenizer_data['merges'])

In [22]:
prompt = 'The best and most reliable form of research is the double-blind, placebo-controlled study.'
print(tokenizer.display(prompt))

tokens = tokenizer.encode(prompt)
print(tokens)

decoded = tokenizer.decode(tokens)
print(decoded)

['The', ' b', 'est', ' and', ' most', ' re', 'l', 'i', 'able', ' for', 'm', ' of', ' re', 'se', 'arch', ' is', ' the', ' d', 'ou', 'ble', '-', 'b', 'l', 'ind', ',', ' pl', 'ace', 'b', 'o', '-', 'c', 'ont', 'ro', 'll', 'ed', ' stud', 'y', '.']
[1021, 442, 277, 394, 288, 790, 303, 108, 105, 570, 320, 109, 287, 303, 362, 739, 326, 263, 293, 278, 808, 45, 98, 108, 523, 44, 469, 456, 98, 111, 45, 99, 783, 302, 599, 280, 930, 121, 46]
The best and most reliable form of research is the double-blind, placebo-controlled study.


In [23]:
for i in range(vocab_size-3):
    print(f"{i}: '{tokenizer.decode([i])}'")

0: ' '
1: ''
2: ''
3: ''
4: ''
5: ''
6: ''
7: ''
8: '
9: '	'
10: '
'
11: ''
12: ''
'3: '
14: ''
15: ''
16: ''
17: ''
18: ''
19: ''
20: ''
21: ''
22: ''
23: ''
24: ''
25: ''
26: ''
27: ''
28: ''
29: ''
30: ''
31: ''
32: ' '
33: '!'
34: '"'
35: '#'
36: '$'
37: '%'
38: '&'
39: '''
40: '('
41: ')'
42: '*'
43: '+'
44: ','
45: '-'
46: '.'
47: '/'
48: '0'
49: '1'
50: '2'
51: '3'
52: '4'
53: '5'
54: '6'
55: '7'
56: '8'
57: '9'
58: ':'
59: ';'
60: '<'
61: '='
62: '>'
63: '?'
64: '@'
65: 'A'
66: 'B'
67: 'C'
68: 'D'
69: 'E'
70: 'F'
71: 'G'
72: 'H'
73: 'I'
74: 'J'
75: 'K'
76: 'L'
77: 'M'
78: 'N'
79: 'O'
80: 'P'
81: 'Q'
82: 'R'
83: 'S'
84: 'T'
85: 'U'
86: 'V'
87: 'W'
88: 'X'
89: 'Y'
90: 'Z'
91: '['
92: '\'
93: ']'
94: '^'
95: '_'
96: '`'
97: 'a'
98: 'b'
99: 'c'
100: 'd'
101: 'e'
102: 'f'
103: 'g'
104: 'h'
105: 'i'
106: 'j'
107: 'k'
108: 'l'
109: 'm'
110: 'n'
111: 'o'
112: 'p'
113: 'q'
114: 'r'
115: 's'
116: 't'
117: 'u'
118: 'v'
119: 'w'
120: 'x'
121: 'y'
122: 'z'
123: '{